## Modulation Transfer Function

In [3]:
#OPTICS and DETECTOR MTF
%matplotlib inline
from ipywidgets import interact,Layout, widgets
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib.cm as cm

def plot_func(Unit,Apt,Dist,Detpitch,WVL1,WVL2):

    skip = False
    if Apt <=0:
        skip=True
    elif Dist <=0:
        skip=True
    elif Detpitch <=0:
        skip=True
    else:
        skip=False

    if skip == True:
        print ('invalid inputs')
    else:
    
        #constants
        pi=math.pi
        numpts = 200  #arbitrary number of points to plot
        fno=Dist/Apt

        #compute cutoffs
        if Unit =="cy/mrad":
            ocutoff = 1/(WVL1*1E-6/(Apt*1e-2)*1000) #cy/mrad
            print ('Cutoff1 =',round(ocutoff,2), 'cy/mrad   for Wavelength = ',WVL1,'um')
            ocutoff2 = 1/(WVL2*1E-6/(Apt*1e-2)*1000) #cy/mrad
            print ('Cutoff2 =',round(ocutoff2,2), 'cy/mrad   for Wavelength = ',WVL2,'um')
        else:    
            ocutoff = round(1/(WVL1*1E-03*fno),2) #cy/mm
            print ('Cutoff1 =',ocutoff, 'cy/mm   for Wavelength = ',WVL1,'um')
            ocutoff2 = round(1/(WVL2*1E-03*fno),2) #cy/mm
            print ('Cutoff2 =',ocutoff2, 'cy/mm   for Wavelength = ',WVL2,'um')

        #set x axis scale for plots 
        step = ocutoff/numpts  #step size 
        lpscale = np.arange(0, ocutoff, step) #line pairs per mm scale = x axis (cy/mm)
        ratio = lpscale/ocutoff
        MTF = lpscale/3 #initialize to any floating point same size as lpscale (divide by 3 was random)

        #calculate the first MTF
        cnt=0
        for i in range(1,numpts+1):
            MTF[cnt]= 2/pi*(math.acos(ratio[cnt]) -  ratio[cnt]*math.sqrt(1-(ratio[cnt])**2))
            #print(MTF[cnt],lpscale[cnt], cnt)
            cnt+=1

        #vertical line at cutoff1
        vlinex = np.repeat(ocutoff, 5)
        vliney = vlinex * 3
        count=0
        for i in vlinex:
            vliney[count] = (count/3)
            count +=1       

        #set plot parameters
        plt.style.use('classic')
        plt.figure(figsize=(16,6))
        plt.subplot(121)    
        #plot optics MTF first wvl

        plt.plot(lpscale, MTF,'bo', label='Wavelength1 = ' + str(WVL1))  # blue
        plt.plot(lpscale, MTF,color='blue')
        plt.text(ocutoff+1, 0.4, " Cutoff1= " + str(round(ocutoff,2)) ,color='blue',rotation=90)
        #plt.text(ocutoff/16,0.13, "F/# = " + str(round(fno,2)) ,color='black')
        plt.plot(vlinex,vliney, 'b:')
        plt.title('Optics Modulation Transfer Function')  
        if Unit == "cy/mm" :
            plt.xlabel('cy/mm')
        else:
            plt.xlabel('cy/mrad')
        plt.ylabel('MTF')
        plt.xlim(0, ocutoff*1.1)
        plt.ylim(0, 1)
        plt.grid(True)

        #Calculate the 2nd MTF
        cnt=0
        ratio3 = lpscale/ocutoff2
        MTF2 = lpscale/3 #use MTF2 for second optics MTF 
        for J in range(1,numpts+1):
            #MTF2[cnt]= 2/pi*(math.acos(ratio2[cnt]) - ratio2[cnt]*math.sqrt(1-(ratio2[cnt])**2))
            if (ratio3[cnt] < 1):
                MTF2[cnt]= 2/pi*(math.acos(ratio3[cnt]) - ratio3[cnt]*math.sqrt(1-(ratio3[cnt])**2))
            else:
                MTF2[cnt]=0.0
            #print(MTF2[cnt],MTF3[cnt],lpscale2[cnt],cnt)
            cnt+=1

        #vertical line at cutoff2
        vlinex = np.repeat(ocutoff2, 5)
        vliney = vlinex * 3
        count=0
        for i in vlinex:
            vliney[count] = (count/3)
            count +=1   
        #overplot second wvl
        plt.plot(lpscale, MTF2,'r*', label='Wavelength2 = ' + str(WVL2))  #red color for second longer wavelenght
        plt.plot(lpscale,MTF2,color='red')
        plt.plot(vlinex,vliney, linestyle='--',color='red')
        plt.text(ocutoff2+1, 0.4, " Cutoff2= " + str(round(ocutoff2,2)) ,color='red',rotation=90)
        plt.legend(loc='best')

        if Unit == "cy/mm":
            #DETECTOR PITCH MTF  - sinc function
            dcutoff = round(1/(Detpitch*1E-03*2),2) #cy/mm
            print ('Nyquist =',dcutoff, 'cy/mm for Detector Pitch =', Detpitch,'um')
            dnumpts = numpts #use sampe as optics MTF = number of step to first zero (1/3 of total)
            dstep = dcutoff/dnumpts
            znumpts = 9 #approx number of detector pitch values which is the z axis  (seems to be 2 less than this)
            zdp = int(znumpts/2) - 1 #target design pitch above.
            dmult = 6 #use 3 to 6X the nyquist freq for plot to show sinc behaviour
            dlpscale = np.arange(0, dmult*dcutoff, dstep)
            #compute detecotr MTF seperately to plot stand alone on its own scale so zero's are captured
            dMTF = dlpscale/5 #detector MTF initialize to any floating point same size as lpscale
            dMTF2 = lpscale/5 #match to lens lpscale
            MTF3D = np.zeros((znumpts, dnumpts)) #MTF's with sweep of detector pitches for 3D plot
            MTF3D2 = np.zeros((znumpts, dnumpts)) #second wvl for above
            #xx=np.zeros((znumpts, dnumpts))#array of detector pitches
            #yy=np.zeros((znumpts, dnumpts))#array of detector pitches
            cntj=1
            for j in range(1, znumpts): 
                cnt=1
                dMTF[0]= 1.0  #sinc function does not like the first zero
                dMTF2[0]= 1.0
                for i in range(0,dmult*dnumpts-1):
                    dMTF[cnt]= abs(math.sin(Detpitch*1E-03*pi*dlpscale[cnt])/(Detpitch*1E-03*pi*dlpscale[cnt]))
                    if (cnt < numpts): #number pf points for optics MTF may differ from Detector numpts 
                        dMTF2[cnt]= abs(math.sin((zdp*Detpitch/cntj)*1E-03*pi*lpscale[cnt])/((zdp*Detpitch/cntj)*1E-03*pi*lpscale[cnt])) #matches lpscale of optics
                        #yy[j][i]= zdp*Detpitch/cnt
                        #xx[j][i]= lpscale[cnt]
                        #print(Detpitch,cnt,cntj,Detpitch/cntj,dMTF[cnt],dMTF2[cnt])               
                    cnt+=1
                #compute composite MTF optics * detector
                if (cntj==zdp): #zdp iteration is the design pitch
                    CMTFW1 = MTF * dMTF2 #compute composite MTF on independant detector dlp scale 
                    CMTFW2 = MTF2 * dMTF2 #second wvl     
                MTF3D[cntj] = dMTF2*MTF #compute composite MTF on optics lp scale
                MTF3D2[cntj] = dMTF2*MTF2
                cntj+=1

            #vertical line at cutoff1
            vlinex = np.repeat(dcutoff, 5)
            vliney = vlinex * 3
            count=0
            for i in vlinex:
                vliney[count] = (count/3)
                count +=1       

            #plot Detector MTF    
            #plt.figure(figsize=(12,6)) 
            plt.subplot(122)
            plt.plot(dlpscale, dMTF,'ko', label='Detector Pitch = ' + str(Detpitch)+ 'um')  # black
            plt.plot(dlpscale, dMTF,color='green')
            plt.text(dcutoff+0.5, 0.4, " Nyquist = " + str(dcutoff) ,color='red',rotation=90)
            plt.plot(vlinex,vliney, linestyle='--',color='red')
            plt.title('Detector Sampling Modulation Transfer Function')  
            if Unit == "cy/mm" :
                plt.xlabel('cy/mm')
            else:
                plt.xlabel('cy/mrad')
            plt.ylabel('MTF')
            plt.xlim(0, dmult*dcutoff)
            plt.ylim(0, 1)
            plt.grid(True)
            plt.legend(loc='best')


            #plot composite MTF for WVL1 and WVL2
            plt.figure(figsize=(14,6))
            plt.plot(lpscale, CMTFW1,'bo', label='WVL1 = ' + str(WVL1)+ 'um')  # blue
            plt.plot(lpscale, CMTFW1,color='blue')
            #plt.text(dcutoff-15, 0.81, " Nyquist = " + str(dcutoff) + " cy/mm",color='red')
            #plt.plot(vlinex,vliney, 'r:')
            plt.plot(lpscale, CMTFW2,'ro', label='WVL2 = ' + str(WVL2)+ 'um')  # red
            plt.plot(lpscale, CMTFW2,color='red')
            plt.text(dcutoff+0.25, 0.05, " Nyquist = " + str(dcutoff) ,color='red',rotation=90)
            plt.plot(vlinex,vliney, linestyle='--',color='red')
            plt.title('Composite Modulation Transfer Function')  
            if Unit == "cy/mm" :
                plt.xlabel('cy/mm')
            else:
                plt.xlabel('cy/mrad')
            plt.ylabel('MTF')
            if dcutoff*2 < ocutoff:
                plt.xlim(0, dcutoff*2.5)
            else:
                plt.xlim(0, ocutoff)
            plt.ylim(0,1)
            #plt.ylim(0.0001, 1)
            #plt.yscale('log')
            plt.grid(True)
            plt.legend(loc='best')


            #rainbow line plot - vary detector pitch plus 4, minus 3
            colors=cm.rainbow(np.linspace(0,1,znumpts))
            plt.style.use('dark_background')
            plt.figure(figsize=(14,6))
            plt.xlim(0,1.1*ocutoff)
            plt.ylim(0, 1)
            plt.grid(True)
            plt.title('MTFs for detector pitch sweep')  
            plt.xlabel('cy/mm')
            plt.ylabel('MTF')
            cntk=1
            for k in range (1, znumpts-1):      
                plt.plot(lpscale,MTF3D[cntk],color=colors[k],linewidth=2.0,alpha=0.80)
                plt.text(ocutoff*.975,0.9-0.7*(k/znumpts),str(int(Detpitch*zdp/k)) +'um',color=colors[k])
                cntk+=1
            plt.plot(lpscale,MTF3D[zdp],linewidth= 3.0,linestyle = ':',label='Detector Pitch = ' + str(Detpitch)+ 'um',color='white')
            if vlinex[0] < ocutoff :
                plt.plot(vlinex,vliney, '--',color='red',linewidth=1.5)

            plt.legend(loc='best')

# Description width style
style1 = {'description_width': 'initial'}
    
interact(plot_func, 
        Unit = ["cy/mm","cy/mrad"],
        Apt  = widgets.FloatText(value=2.0, step=0.1,description ="Aperture     (cm)",style=style1),
        Dist = widgets.FloatText(value=6.0, step=1,description ="Distance   (cm)",style=style1),
        Detpitch = widgets.FloatText(value=35, step=1,description = "Det Pitch (um)",style=style1),
        WVL1 = widgets.FloatSlider(value=3, min=1, max=15, step=0.25,description="WVL1 (" + chr(955) + " um)",layout=Layout(width='400px'),continuous_update=False,style=style1),
        WVL2 = widgets.FloatSlider(value=5, min=1, max=15, step=0.25,description="WVL2 (" + chr(955) + " um)",layout=Layout(width='400px'),continuous_update=False,style=style1)
        )





interactive(children=(Dropdown(description='Unit', options=('cy/mm', 'cy/mrad'), value='cy/mm'), FloatText(val…

<function __main__.plot_func(Unit, Apt, Dist, Detpitch, WVL1, WVL2)>